# Facial recognition and trigger

Installing necessary libraries

In [1]:
pip install opencv-python opencv-contrib-python pillow numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


import necessary libraries

In [42]:
import os
import json
import cv2
import numpy as np
from PIL import Image

creating necessary folders

In [3]:
mkdir -p FacialRecognitionProject dataset trainer

### Creating some utility functions

In [49]:
def save_to_disk(face_dict, path="faces_lib.json"):
    if os.path.exists(path):  # read existing data
        with open(path, "r") as f:
            data = json.load(f)
    else:
        data = {}
    with open(path, "w+") as f:  # add new data
        data.update(face_dict)
        json.dump(data, f)  # save to disk


def get_faces_lib(path="faces_lib.json"):
    if os.path.exists(path):  # read existing data
        with open(path, "r") as f:
            data = json.load(f)
    else:
        data = {}
    return data


## Registering a face

In [50]:
def register_face():
    """
    register a face and save 30 images of the face to disk
    """
    cam = cv2.VideoCapture(0)
    cam.set(3, 640)  # set video width
    cam.set(4, 480)  # set video height

    face_detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    # For each person, enter one numeric face id
    face_id = input("\n enter user id and press <enter> ==>  ")
    name = input("\n enter user's name and press <enter> ==>  ")
    print("\n [INFO] Initializing face capture. Look the camera and wait ...")
    # Initialize individual sampling face count
    count = 0
    while True:
        ret, img = cam.read()
        img = cv2.flip(img, 1)  # flip video image vertically
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            count += 1
            # Save the captured image into the datasets folder
            cv2.imwrite(
                f"dataset/User.{face_id}.{count}.jpg",
                gray[y : y + h, x : x + w],
            )
            cv2.imshow("image", img)

        k = cv2.waitKey(100) & 0xFF  # Press 'ESC' for exiting video
        if k == 27:
            break
        elif count >= 30:  # Take 30 face sample and stop video
            break
    # Do a bit of cleanup
    print("\n [INFO] Exiting Program and cleanup stuff")
    cam.release()
    cv2.destroyAllWindows()
    # save the name of the person
    save_to_disk({face_id: name})


In [51]:
register_face()


 [INFO] Initializing face capture. Look the camera and wait ...

 [INFO] Exiting Program and cleanup stuff


## Training the model

train our model to recognise the registered faces

In [53]:
# Path for face image database
path = "dataset"
recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
# function to get the images and label data
def getImagesAndLabels(path):
    imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
    faceSamples = []
    ids = []
    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert("L")  # grayscale
        img_numpy = np.array(PIL_img, "uint8")
        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faces = detector.detectMultiScale(img_numpy)
        for (x, y, w, h) in faces:
            faceSamples.append(img_numpy[y : y + h, x : x + w])
            ids.append(id)
    return faceSamples, ids


print("\n [INFO] Training faces. It will take a few seconds. Wait ...")

faces, ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))
# Save the model into trainer/trainer.yml
recognizer.write("trainer/trainer.yml")
print(f"\n [INFO] {len(np.unique(ids))} faces trained. Exiting Program")



 [INFO] Training faces. It will take a few seconds. Wait ...

 [INFO] 1 faces trained. Exiting Program
